# **Nayab Irfan | RouteForge**

---



```
> AgenticAI Cohort 2 | Assignment 1
Assignment: AI Research & Planning Agent | Travel Agent

```



**Methodolody → find places → optimize route cost → maximize fun → arrive**
###### Tools used:
 - Tavily (primary) & SerpAPI (fallback) for search (last 12 months intent)
 - SerpAPI Google Maps (if key present) for places; Overpass (OSM) fallback
 - OSRM (public) for routing & distance/time (no key required)
 - OpenAI LLM for itinerary (Groq fallback)
 - LangChain tools + tool-calling agent to show multi-step planning

 Outputs:
 - Report.md  (final trip plan)
 - trip_plan.json (all sources, chosen places, route, costs)


#### **INSTALLS**



In [148]:
!pip install langchain-openai langchain-groq
!pip -q install --upgrade pip
!pip -q install \
  langchain==0.2.13 \
  langchain-openai==0.1.23 \
  langchain-groq==0.1.3 \
  langchain-community==0.2.9 \
  tavily-python==0.3.6 \
  serpapi==0.1.5 \
  requests==2.32.3 \
  tiktoken==0.7.0

ERROR: Cannot install langchain-groq==0.1.3, langchain-openai==0.1.23 and langchain==0.2.13 because these package versions have conflicting dependencies.
ERROR: ResolutionImpossible: for help visit https://pip.pypa.io/en/latest/topics/dependency-resolution/#dealing-with-dependency-conflicts


In [149]:
!pip install tavily-python

#### **1) IMPORTS & SETUP**

In [150]:
import os, json, time, math, re, traceback, datetime, textwrap, random
from typing import List, Dict, Any, Optional, Tuple

import requests

from langchain.tools import tool
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq

from tavily import TavilyClient

#### **2) API KEYS**

In [ ]:
os.environ["OPENAI_API_KEY"] = "enter yours"
os.environ["GROQ_API_KEY"]   = "enter yours"
os.environ["TAVILY_API_KEY"] = "enter yours"
os.environ["SERPAPI_API_KEY"]= "enter yours"  # optional fallback


#### **3) GLOBAL CONFIG**

In [152]:
OUTPUT_MD   = "Report.md"
OUTPUT_JSON = "trip_plan.json"

SEARCH_QUERIES_TMPL = [
    "best things to do in {city} 2024 2025",
    "best restaurants in {city} 2024 2025",
    "free activities in {city} 2024 2025",
    "hidden gems in {city} 2024 2025",
    "local food specialties in {city} 2024 2025"
]

DEFAULTS = {
    "place_radius_m": 4000,               # search radius around destination center
    "max_candidates": 30,                 # max places pulled per category
    "top_k": 8,                           # final number of stops (including food & fun) before the end destination
    "transport_mode": "driving",          # driving | walking | cycling
    "cost_per_km": 0.25,                  # simple cost model (fuel/transport) — tweak as you like
    "time_value_per_hr": 5.0              # optional time value ($/hr) to penalize long routes
}

STATE = {
    "inputs": {},
    "search_sources": [], # Tavily links (context)
    "places": [],         # unified candidates (from SerpAPI or Overpass)
    "route": {},          # solution with ordering/distances/cost
    "itinerary_md": ""
}


#### **4) LLM HELPERS (OpenAI primary, Groq fallback elsewhere)**

In [153]:
def get_openai_llm(model="gpt-4o-mini", temperature=0):
    return ChatOpenAI(model=model, temperature=temperature, timeout=120)

def get_groq_llm(model="llama-3.1-8b-instant", temperature=0):
    return ChatGroq(model_name=model, temperature=temperature, timeout=120)

def get_groq2_llm(model="llama3-70b-8192", temperature=0):
    return ChatGroq(model_name=model, temperature=temperature, timeout=120)

def llm_call(messages, openai_model="gpt-4o-mini", groq_model="llama-3.1-8b-instant", groq2_model="llama3-70b-8192"):
    """
    Generic LLM call for summaries/extraction: try OpenAI, then Groq, then Groq-70B.
    (Your tool-calling agent still uses OpenAI inside run_agent.)
    """
    try:
        return get_openai_llm(openai_model).invoke(messages).content
    except Exception as e1:
        print("OpenAI failed, falling back to Groq:", e1)
        try:
            return get_groq_llm(groq_model).invoke(messages).content
        except Exception as e2:
            print("Groq 8B failed, falling back to Groq 70B:", e2)
            try:
                return get_groq2_llm(groq2_model).invoke(messages).content
            except Exception as e3:
                print("All LLMs failed:", e3)
                return None

#### **5) BASIC GEO UTILITIES (Geocoding & Helpers)**

In [154]:
def geocode_nominatim(q: str) -> Optional[Tuple[float,float,str]]:
    # OpenStreetMap Nominatim (no key; be nice)
    url = "https://nominatim.openstreetmap.org/search"
    params = {"q": q, "format": "json", "limit": 1}
    headers = {"User-Agent": "Colab-Travel-Agent/1.0"}
    try:
        r = requests.get(url, params=params, headers=headers, timeout=20)
        r.raise_for_status()
        data = r.json()
        if data:
            lat = float(data[0]["lat"]); lon = float(data[0]["lon"])
            disp = data[0].get("display_name", q)
            return lat, lon, disp
    except Exception as e:
        print("geocode_nominatim error:", e)
    return None

def haversine_km(a: Tuple[float,float], b: Tuple[float,float]) -> float:
    R = 6371.0
    lat1, lon1 = map(math.radians, a)
    lat2, lon2 = map(math.radians, b)
    dlat = lat2 - lat1; dlon = lon2 - lon1
    h = math.sin(dlat/2)**2 + math.cos(lat1)*math.cos(lat2)*math.sin(dlon/2)**2
    return 2*R*math.asin(math.sqrt(h))

def multi_geocode(query: str):
    """
    Try plain Nominatim, then city-biased, then Photon.
    Returns (lat, lon, label) or None.
    """
    # 1) plain Nominatim
    hit = geocode_nominatim(query)
    if hit:
        return hit
    # 2) Photon fallback
    hit = geocode_photon(query)
    if hit:
        return hit
    return None

In [155]:
# ======================
# 5.1) CITY-BIASED GEOCODING HELPERS (Nominatim + Photon fallback)
# ======================
import math

_GEO_UA = {"User-Agent": "Colab-Travel-Agent/1.1 (+no-keys)"}

def _bbox_from_center(lat: float, lon: float, box_km: float = 12.0):
    """Return (lon_min, lat_min, lon_max, lat_max) for a square ~box_km around center."""
    dlat = box_km / 111.0
    dlon = box_km / (111.0 * max(0.1, math.cos(math.radians(lat))))
    return (lon - dlon, lat - dlat, lon + dlon, lat + dlat)

def geocode_nominatim_boxed(query: str, center_lat: float, center_lon: float, box_km: float = 12.0):
    """Nominatim geocoding biased to a city area via viewbox + bounded=1."""
    lon_min, lat_min, lon_max, lat_max = _bbox_from_center(center_lat, center_lon, box_km)
    url = "https://nominatim.openstreetmap.org/search"
    params = {
        "q": query,
        "format": "json",
        "limit": 1,
        "viewbox": f"{lon_min},{lat_min},{lon_max},{lat_max}",
        "bounded": 1,
        "addressdetails": 0,
    }
    try:
        r = requests.get(url, params=params, headers=_GEO_UA, timeout=20)
        r.raise_for_status()
        data = r.json()
        if data:
            return float(data[0]["lat"]), float(data[0]["lon"]), data[0].get("display_name", query)
    except Exception as e:
        print("geocode_nominatim_boxed error:", e)
    return None

def geocode_photon(query: str):
    """Photon (Komoot) fallback — no key."""
    url = "https://photon.komoot.io/api/"
    try:
        r = requests.get(url, params={"q": query, "limit": 1}, headers=_GEO_UA, timeout=20)
        r.raise_for_status()
        js = r.json()
        feats = js.get("features") or []
        if feats:
            coords = feats[0]["geometry"]["coordinates"]  # [lon, lat]
            lat, lon = float(coords[1]), float(coords[0])
            label = feats[0]["properties"].get("name", query)
            return lat, lon, label
    except Exception as e:
        print("geocode_photon error:", e)
    return None

def geocode_in_city(query_fragment: str, city_center: tuple, box_km: float = 12.0):
    """
    Resolve a short/vague spot (e.g., 'liberty market') within the city
    by biasing geocoding to a viewbox around the city center.
    """
    latc, lonc = city_center
    hit = geocode_nominatim_boxed(query_fragment, latc, lonc, box_km=box_km)
    if hit:
        return hit
    return geocode_photon(query_fragment)


#### **6) SEARCH (Tavily) — context | guides about destination**

In [156]:
def tavily_search_city(city: str, k: int = 5) -> List[Dict[str, str]]:
    out = []
    try:
        tv = TavilyClient(api_key=os.environ.get("TAVILY_API_KEY",""))
        for q in SEARCH_QUERIES_TMPL:
            query = q.format(city=city)
            res = tv.search(query=query, search_depth="advanced", include_answer=False, max_results=max(3, k), days=365)
            for item in res.get("results", []):
                out.append({"title": item.get("title",""), "url": item.get("url",""), "content": (item.get("content","") or "")[:400]})
            time.sleep(0.3)
    except Exception as e:
        print("Tavily search failed:", e)
    return out[:k*len(SEARCH_QUERIES_TMPL)]


#### **7) PLACES DISCOVERY**
*    *SerpAPI Google Maps (preferred if key provided)*
*    *Overpass (OSM) fallback*

In [157]:
def serpapi_maps_places(query: str, lat: float, lon: float, radius_m: int, max_n: int) -> List[Dict[str, Any]]:
    api_key = os.environ.get("SERPAPI_API_KEY","")
    if not api_key or "YOUR_" in api_key:
        return []
    url = "https://serpapi.com/search.json"
    params = {
        "engine": "google_maps",
        "type": "search",
        "q": query,
        "ll": f"@{lat},{lon},15z",
        "api_key": api_key
    }
    try:
        r = requests.get(url, params=params, timeout=30)
        r.raise_for_status()
        data = r.json()
        results = data.get("local_results", []) or data.get("places", [])
        out = []
        for x in results[:max_n]:
            # Standardize fields
            name = x.get("title") or x.get("name")
            rating = x.get("rating") or x.get("gps_coordinates",{}).get("rating") or x.get("rating_score")
            review_cnt = x.get("reviews") or x.get("user_ratings_total") or x.get("reviews_count")
            address = x.get("address") or x.get("address_lines") or x.get("full_address")
            coords = x.get("gps_coordinates") or {}
            plat, plon = coords.get("latitude"), coords.get("longitude")
            price_level = x.get("price") or x.get("price_level")
            link = x.get("link") or x.get("website") or x.get("cid")
            types = x.get("type") or x.get("types") or []
            out.append({
                "name": name, "lat": plat, "lon": plon, "rating": rating, "reviews": review_cnt,
                "price_level": price_level, "address": address, "types": types,
                "source": "serpapi_google_maps", "url": link or ""
            })
        return [p for p in out if p["lat"] and p["lon"]]
    except Exception as e:
        print("serpapi_maps_places error:", e)
        return []

def overpass_query(lat: float, lon: float, radius_m: int, query: str) -> List[Dict[str, Any]]:
    # query can be 'restaurant' or 'attraction'
    if query == "restaurant":
        q = f"""
        [out:json];
        node(around:{radius_m},{lat},{lon})[amenity=restaurant];
        out center 50;
        """
    else:
        # "things to do" proxy: tourism=attraction + sightseeing: yes
        q = f"""
        [out:json];
        (
          node(around:{radius_m},{lat},{lon})[tourism=attraction];
          node(around:{radius_m},{lat},{lon})[amenity=park];
          node(around:{radius_m},{lat},{lon})[leisure=park];
        );
        out center 70;
        """
    try:
        r = requests.post("https://overpass-api.de/api/interpreter", data=q, timeout=60)
        r.raise_for_status()
        data = r.json()
        out = []
        for e in data.get("elements", []):
            tags = e.get("tags", {})
            out.append({
                "name": tags.get("name") or "Unnamed",
                "lat": e.get("lat"), "lon": e.get("lon"),
                "rating": None, "reviews": None,
                "price_level": None,
                "address": tags.get("addr:full") or "",
                "types": [tags.get("amenity") or tags.get("tourism") or tags.get("leisure")],
                "source": "overpass_osm",
                "url": f"https://www.openstreetmap.org/node/{e.get('id')}"
            })
        return [p for p in out if p["lat"] and p["lon"]]
    except Exception as e:
        print("overpass error:", e)
        return []

def get_places(lat: float, lon: float, radius_m: int, max_n: int) -> List[Dict[str,Any]]:
    out = []
    # Prefer SerpAPI if available
    ser_ok = os.environ.get("SERPAPI_API_KEY","") and "YOUR_" not in os.environ.get("SERPAPI_API_KEY","")
    if ser_ok:
        out += serpapi_maps_places("things to do", lat, lon, radius_m, max_n)
        out += serpapi_maps_places("restaurants",  lat, lon, radius_m, max_n)
    # Fallback to Overpass to ensure we have something
    if len(out) < 10:
        out += overpass_query(lat, lon, radius_m, "attraction")
        out += overpass_query(lat, lon, radius_m, "restaurant")
    # de-dup by name+coords
    seen = set()
    uniq = []
    for p in out:
        key = (p["name"], round(p["lat"],5), round(p["lon"],5))
        if key not in seen:
            seen.add(key)
            uniq.append(p)
    return uniq[:max_n*2]


In [158]:
# ======================
# 7.1) SPECIFIC POI FINDER (café / pharmacy / restroom / cuisine or anything!) via Overpass
# ======================
import re

def find_specific_places(center_lat: float, center_lon: float, radius_m: int, query_text: str):
    """
    Returns list of {name, lat, lon, category, address, url}
    Supports amenity keywords and cuisine (e.g. 'italian', 'pakistani').
    """
    if not query_text:
        return []

    txt = query_text.lower().strip()
    synonyms = {
        "cafe": ["cafe", "coffee", "coffee shop", "chai"],
        "market": ["bazar", "grocery", "shop stop", "mart"],
        "fuel": ["CNG", "petrol pump", "gas station"],
        "pharmacy": ["pharmacy", "hospital", "clinic"],
        "restroom": ["restroom", "toilet", "washroom", "toilets", "bathroom"],
        "restaurant": ["restaurant", "eatery", "food", "diner", "lunch", "takeaway"],
    }
    amenity = None
    cuisine = None

    tokens = [t for t in re.split(r"[,/; ]+", txt) if t]
    for k, words in synonyms.items():
        if any(w in tokens for w in words):
            amenity = {"cafe":"cafe", "pharmacy":"pharmacy", "restroom":"toilets", "restaurant":"restaurant"}[k]
            break

    cuisine_words = {"pakistani","italian","chinese","japanese","thai","indian","turkish","korean","mexican","french","arabic","lebanese","bbq","seafood","vegan","vegetarian","bakery","dessert"}
    guessed = [w for w in tokens if w in cuisine_words]
    if guessed:
        cuisine = "|".join(guessed)
        if amenity is None:
            amenity = "restaurant"

    if amenity is None and not cuisine:
        amenity = "restaurant"
        cuisine = "|".join(tokens)  # treat unknown words as cuisine pattern

    filters = []
    if amenity:
        filters.append(f'[amenity="{amenity}"]')
    if cuisine:
        filters.append(f'[cuisine~"{cuisine}",i]')

    overpass = f"""
    [out:json][timeout:60];
    (
      node(around:{radius_m},{center_lat},{center_lon}){''.join(filters)};
      way(around:{radius_m},{center_lat},{center_lon}){''.join(filters)};
      relation(around:{radius_m},{center_lat},{center_lon}){''.join(filters)};
    );
    out center 100;
    """
    try:
        r = requests.post("https://overpass-api.de/api/interpreter", data=overpass, timeout=90)
        r.raise_for_status()
        data = r.json()
        out = []
        for e in data.get("elements", []):
            tags = e.get("tags", {}) or {}
            lat = e.get("lat") or (e.get("center", {}) or {}).get("lat")
            lon = e.get("lon") or (e.get("center", {}) or {}).get("lon")
            if not (lat and lon):
                continue
            name = tags.get("name") or (cuisine if cuisine else amenity) or "Unnamed"
            address = tags.get("addr:full") or ""
            url = f"https://www.openstreetmap.org/{e.get('type','node')}/{e.get('id')}"
            out.append({
                "name": name,
                "lat": float(lat),
                "lon": float(lon),
                "category": "specific",
                "address": address,
                "url": url
            })
        return out
    except Exception as e:
        print("find_specific_places error:", e)
        return []


In [159]:
# ===== 7.3) Execute parsed intents: SerpAPI (Google Maps) -> Nominatim/Overpass fallback =====

def _serpapi_maps_search(q: str, lat: float, lon: float, max_n: int = 10):
    api_key = os.environ.get("SERPAPI_API_KEY","")
    if not api_key:
        return []
    url = "https://serpapi.com/search.json"
    params = {
        "engine": "google_maps",
        "type": "search",
        "q": q,
        "ll": f"@{lat},{lon},15z",
        "api_key": api_key
    }
    try:
        r = requests.get(url, params=params, timeout=30)
        r.raise_for_status()
        data = r.json()
        results = data.get("local_results", []) or data.get("places", [])
        out = []
        for x in results[:max_n]:
            name = x.get("title") or x.get("name") or q
            coords = x.get("gps_coordinates") or {}
            plat, plon = coords.get("latitude"), coords.get("longitude")
            addr = x.get("address") or x.get("address_lines") or ""
            link = x.get("link") or x.get("website") or ""
            types = x.get("type") or x.get("types") or []
            if plat and plon:
                out.append({
                    "name": name, "lat": float(plat), "lon": float(plon),
                    "category": "specific", "address": addr, "types": types,
                    "source": "serpapi_google_maps", "url": link
                })
        return out
    except Exception as e:
        print("SerpAPI search error:", e)
        return []

def _nominatim_box_search(q: str, center_xy, box_km: float = 15.0, limit: int = 10):
    # bias Nominatim to the city area via viewbox
    latc, lonc = center_xy
    def _bbox(lat, lon, box_km):
        dlat = box_km / 111.0
        dlon = box_km / (111.0 * max(0.1, math.cos(math.radians(lat))))
        return (lon - dlon, lat - dlat, lon + dlon, lat + dlat)
    lon_min, lat_min, lon_max, lat_max = _bbox(latc, lonc, box_km)
    url = "https://nominatim.openstreetmap.org/search"
    params = {
        "q": q, "format": "json", "limit": limit,
        "viewbox": f"{lon_min},{lat_min},{lon_max},{lat_max}",
        "bounded": 1
    }
    try:
        r = requests.get(url, params=params, headers={"User-Agent":"TravelAgent/1.2"}, timeout=30)
        r.raise_for_status()
        rows = r.json() or []
        out = []
        for row in rows:
            out.append({
                "name": row.get("display_name", q),
                "lat": float(row["lat"]), "lon": float(row["lon"]),
                "category": "specific", "address": row.get("display_name",""),
                "types": [], "source": "nominatim", "url": f"https://www.openstreetmap.org/{row.get('osm_type','node')}/{row.get('osm_id','')}"
            })
        return out
    except Exception as e:
        print("Nominatim search error:", e)
        return []

def _overpass_osm_tag(center_xy, radius_m, key: str, value: str, limit: int = 40):
    lat, lon = center_xy
    overpass = f"""
    [out:json][timeout:60];
    (
      node(around:{radius_m},{lat},{lon})[{key}="{value}"];
      way(around:{radius_m},{lat},{lon})[{key}="{value}"];
      relation(around:{radius_m},{lat},{lon})[{key}="{value}"];
    );
    out center {limit};
    """
    try:
        r = requests.post("https://overpass-api.de/api/interpreter", data=overpass, timeout=90)
        r.raise_for_status()
        data = r.json() or {}
        out = []
        for e in data.get("elements", []):
            tags = e.get("tags", {}) or {}
            plat = e.get("lat") or (e.get("center", {}) or {}).get("lat")
            plon = e.get("lon") or (e.get("center", {}) or {}).get("lon")
            if not (plat and plon):
                continue
            name = tags.get("name") or value
            url = f"https://www.openstreetmap.org/{e.get('type','node')}/{e.get('id')}"
            out.append({
                "name": name, "lat": float(plat), "lon": float(plon),
                "category": "specific", "address": tags.get("addr:full",""),
                "types": [f"{key}={value}"], "source": "overpass_osm", "url": url
            })
        return out
    except Exception as e:
        print("Overpass (OSM tag) error:", e)
        return []

def resolve_parsed_specific_requests(center_xy, radius_m, parsed_items, max_per_item: int = 10):
    """
    For each parsed item:
      - 'place'/'area': SerpAPI → Nominatim (city-biased) → multi_geocode (guarantee 1)
      - 'category' (with optional cuisine): SerpAPI → Nominatim
      - 'osm' tag: Overpass direct
    Returns normalized places list (each tagged with category='specific').
    """
    out = []
    latc, lonc = center_xy

    for it in (parsed_items or []):
        t = (it.get("type") or "").lower()
        if t == "place":
            q = it.get("name","").strip()
            if not q:
                continue
            # 1) SerpAPI
            got = _serpapi_maps_search(q, latc, lonc, max_n=max_per_item)
            # 2) Nominatim (city box)
            if not got:
                got = _nominatim_box_search(q, center_xy, limit=max_per_item)
            # 3) multi_geocode (guarantee at least one point)
            if not got:
                hit = geocode_in_city(q, center_xy) or multi_geocode(q)
                if hit:
                    plat, plon, label = hit
                    got = [{
                        "name": label, "lat": float(plat), "lon": float(plon),
                        "category": "specific", "address": label,
                        "types": ["user_specified","place"],
                        "source": "geocode_fallback", "url": ""
                    }]
            # tag & add
            for p in (got or []):
                p.setdefault("category","specific")
            out.extend(got or [])

        elif t == "area":
            q = it.get("name","").strip()
            if not q:
                continue
            got = _nominatim_box_search(q, center_xy, limit=max_per_item)
            for p in (got or []):
                p.setdefault("category","specific")
            out.extend(got or [])

        elif t == "category":
            lbl = (it.get("label") or "").strip()
            cuisine = (it.get("cuisine") or "").strip()
            if not lbl:
                continue
            q = f"{cuisine} {lbl}".strip() if cuisine else lbl
            got = _serpapi_maps_search(q, latc, lonc, max_n=max_per_item) or _nominatim_box_search(q, center_xy, limit=max_per_item)
            for p in (got or []):
                p.setdefault("category","specific")
            out.extend(got or [])

        elif t == "osm":
            key = (it.get("key") or "").strip(); value = (it.get("value") or "").strip()
            if key and value:
                got = _overpass_osm_tag(center_xy, radius_m, key, value, limit=max_per_item)
                for p in (got or []):
                    p.setdefault("category","specific")
                out.extend(got or [])

    # de‑dup by (name, lat, lon)
    dedup, seen = [], set()
    for p in out:
        key = (p.get("name",""), round(float(p.get("lat",0) or 0),5), round(float(p.get("lon",0) or 0),5))
        if key not in seen:
            seen.add(key); dedup.append(p)
    return dedup

#### **8) OSRM ROUTING (distance/time & route)**

In [160]:
def osrm_table(coords: List[Tuple[float,float]], mode="driving") -> Dict[str, Any]:
    base = f"https://router.project-osrm.org/table/v1/{mode}/"
    path = ";".join([f"{lon},{lat}" for lat,lon in coords])
    url = base + path
    try:
        r = requests.get(url, params={"annotations":"duration,distance"}, timeout=60)
        r.raise_for_status()
        # OSRM returns null for unreachable destinations. Need to handle this.
        data = r.json()
        # Replace None in distances/durations with a large number (or handle later)
        dist_matrix = data.get("distances")
        dur_matrix = data.get("durations")

        if dist_matrix:
            for row in dist_matrix:
                for i in range(len(row)):
                    if row[i] is None:
                        row[i] = float('inf') # Treat unreachable as infinite distance
        if dur_matrix:
            for row in dur_matrix:
                 for i in range(len(row)):
                     if row[i] is None:
                         row[i] = float('inf') # Treat unreachable as infinite duration # Use inf for duration too

        return {"distances": dist_matrix, "durations": dur_matrix}
    except Exception as e:
        print("osrm_table error:", e)
        return {}

def osrm_route_summary(a: Tuple[float,float], b: Tuple[float,float], mode="driving") -> Optional[Tuple[float,float]]:
    base = f"https://router.project-osrm.org/route/v1/{mode}/"
    url  = base + f"{a[1]},{a[0]};{b[1]},{b[0]}"
    try:
        r = requests.get(url, params={"overview":"false","alternatives":"false","steps":"false","annotations":"false"}, timeout=60)
        r.raise_for_status()
        data = r.json()
        routes = data.get("routes", [])
        if routes:
            dist_m = routes[0].get("distance", 0.0)
            dur_s  = routes[0].get("duration", 0.0)
            return dist_m, dur_s
    except Exception as e:
        print("osrm_route_summary error:", e)
    return None

# Simple greedy TSP (start fixed at origin, end fixed at destination)
def plan_route(origin: Tuple[float,float], dest: Tuple[float,float], stops: List[Dict[str,Any]], mode="driving") -> Dict[str,Any]:
    # Build coords: origin + stops + dest
    coords = [origin] + [(p["lat"], p["lon"]) for p in stops if p and p.get("lat") is not None and p.get("lon") is not None] + [dest]

    # Ensure at least origin and destination are present
    if len(coords) < 2:
        print("plan_route: Not enough valid coordinates for routing.")
        # Return a minimal valid route structure indicating failure or no stops
        return {"order": [0, 1], "legs": [{"from_index": 0, "to_index": 1, "distance_m": 0.0, "duration_s": 0.0}], "total_distance_m": 0.0, "total_duration_s": 0.0}


    table = osrm_table(coords, mode=mode)
    dist = table.get("distances")
    dur  = table.get("durations")

    # Fallback to pairwise haversine if OSRM table failed or returned no data
    if not dist or not dur or any(any(x is None for x in row) for row in dist) or any(any(x is None for x in row) for row in dur):
        print("OSRM table failed or returned None/invalid data. Falling back to Haversine.")
        n = len(coords)
        dist = [[0.0]*n for _ in range(n)]
        dur  = [[0.0]*n for _ in range(n)]
        speed_kmh = 40 if mode=="driving" else (5 if mode=="walking" else 15)
        for i in range(n):
            for j in range(n):
                if i==j: continue
                try:
                    d = haversine_km(coords[i], coords[j])
                    dist[i][j] = d*1000.0 # Convert to meters
                    dur[i][j]  = (d/speed_kmh)*3600.0 # Convert to seconds
                except Exception as e:
                    print(f"Haversine calculation failed for {coords[i]} to {coords[j]}: {e}")
                    dist[i][j] = float('inf')
                    dur[i][j] = float('inf')


    # Ensure dist and dur are lists of lists of numbers/inf
    if not isinstance(dist, list) or not all(isinstance(row, list) for row in dist) or \
       not isinstance(dur, list) or not all(isinstance(row, list) for row in dur):
        print("Fallback Haversine calculation failed to produce valid matrices.")
        # Return minimal failed route
        return {"order": [0, len(coords)-1] if len(coords)>1 else [0], "legs": [], "total_distance_m": float('inf'), "total_duration_s": float('inf')}


    # greedy: from origin (0) to choose next closest among stops, end at last index (n-1) enforced
    n = len(coords)
    must_end = n-1
    # Indices of stops in the coords list are from 1 to n-2
    unvisited_indices_in_coords = set(range(1, n-1))
    route_indices_in_coords = [0] # Start at origin (index 0 in coords)

    curr_idx_in_coords = 0 # Start at origin

    while unvisited_indices_in_coords:
        # Find the next unvisited index in coords that minimizes distance from current
        # Safely handle potential None or non-numeric in dist matrix lookup
        try:
            next_idx_in_coords = min(
                unvisited_indices_in_coords,
                key=lambda j: dist[curr_idx_in_coords][j] if isinstance(dist[curr_idx_in_coords][j], (int, float)) else float('inf')
            )
            route_indices_in_coords.append(next_idx_in_coords)
            unvisited_indices_in_coords.remove(next_idx_in_coords)
            curr_idx_in_coords = next_idx_in_coords
        except Exception as e:
            print(f"Error during greedy TSP next step selection: {e}. Remaining unvisited: {unvisited_indices_in_coords}")
            # Break out of loop if min operation fails
            break

    # Ensure the route ends at the destination (must_end)
    if route_indices_in_coords[-1] != must_end:
         # If the last point added wasn't the destination, add it now
         if must_end not in route_indices_in_coords:
              route_indices_in_coords.append(must_end)
         else:
             # If destination was visited somewhere in the middle, ensure it's also the final stop
             # This simple greedy doesn't handle visiting the end stop mid-route well.
             # For robustness, if destination was visited, we might simplify the route to just origin -> destination
             # Or, if it's not the last point, remove intermediate points after its first visit and add it again at the end.
             # Simplest: if destination is in route but not last, make route origin -> destination.
             # More complex: find last unvisited point, connect it to destination.
             # Let's stick to simple: if the destination is in the route but not the end, force origin->destination route as a safe path.
             if route_indices_in_coords[-1] != must_end: # Double check
                print("Destination visited but not as final stop. Forcing origin -> destination route.")
                route_indices_in_coords = [0, must_end] # Simplest valid route

    # accumulate totals
    legs = []
    total_dist_m = 0.0
    total_dur_s  = 0.0
    for i in range(len(route_indices_in_coords)-1):
        a, b = route_indices_in_coords[i], route_indices_in_coords[i+1]
        # Ensure distance/duration are numbers before accumulating
        dist_val = dist[a][b] if isinstance(dist[a][b], (int, float)) else float('inf')
        dur_val = dur[a][b] if isinstance(dur[a][b], (int, float)) else float('inf')

        total_dist_m += dist_val
        total_dur_s  += dur_val
        legs.append({"from_index": a, "to_index": b, "distance_m": dist_val, "duration_s": dur_val})

    # Map coords indices back to original "order" which includes origin/stops/destination
    # The order needs to reference the indices in the original `coords` list (0 for origin, 1...n-2 for stops, n-1 for dest)
    final_order_indices = route_indices_in_coords

    return {"order": final_order_indices, "legs": legs, "total_distance_m": total_dist_m, "total_duration_s": total_dur_s}

#### **9) FUN SCORE & SELECTION**

In [161]:
def normalize(x, lo, hi):
    if hi <= lo: return 0.0
    x = max(lo, min(hi, x))
    return (x - lo) / (hi - lo)

def compute_fun_scores(places: List[Dict[str,Any]], center: Tuple[float,float]) -> List[Dict[str,Any]]:
    max_reviews = max([p.get("reviews", 0) or 0 for p in places] + [1]) # Use .get() here

    # map price symbols to numeric
    def price_to_num(v):
        if v is None: return 2
        s = str(v)
        if s.isdigit(): return int(s)
        return s.count("$") if "$" in s else 2

    scored = []
    for p in places:
        # Use .get() with default values
        rating = float(p.get("rating", 0.0)) if p.get("rating") not in [None,""] else 0.0
        reviews = float(p.get("reviews", 0)) if p.get("reviews") not in [None,""] else 0.0 # Use .get() here
        price = price_to_num(p.get("price_level")) # Use .get() here
        # Ensure lat/lon are valid before computing distance
        if p.get("lat") is None or p.get("lon") is None:
            dist_km = float('inf') # Or handle as needed, setting to inf makes it less likely to be picked
        else:
            dist_km = haversine_km(center, (p["lat"], p["lon"]))

        s_rating  = normalize(rating, 3.5, 5.0)           # favor 4.0+
        s_pop     = normalize(math.log1p(reviews), 0, math.log1p(max_reviews))
        s_close   = 1 - normalize(dist_km, 0, 8)          # closer is better
        s_price   = 1 - normalize(price, 1, 4)            # cheaper is better

        fun = 0.46*s_rating + 0.32*s_pop + 0.14*s_close + 0.08*s_price
        p2 = dict(p); p2["fun_score"] = round(fun, 4); p2["dist_from_center_km"] = round(dist_km, 2)
        scored.append(p2)

    # diversify by type: interleave top N from two buckets (food vs fun)
    food  = [p for p in scored if any(t for t in (p.get("types") or []) if "restaurant" in str(t).lower() or "food" in str(t).lower())]
    funs  = [p for p in scored if p not in food]
    food.sort(key=lambda x: x["fun_score"], reverse=True)
    funs.sort(key=lambda x: x["fun_score"], reverse=True)

    return food, funs, scored

#### **10) LANGCHAIN TOOLS**

In [162]:
@tool("geo_destination")
def geo_destination_tool(query: str) -> dict:
    """Geocode a destination or address into latitude/longitude and a nice display name."""
    res = geocode_nominatim(query)
    return {"ok": bool(res), "lat": res[0] if res else None, "lon": res[1] if res else None, "label": res[2] if res else None}

@tool("discover_places")
def discover_places_tool(lat: float, lon: float, radius_m: int = DEFAULTS["place_radius_m"], max_candidates: int = DEFAULTS["max_candidates"]) -> dict:
    """Find restaurants and attractions near the destination using SerpAPI (if available) with Overpass fallback."""
    places = get_places(lat, lon, radius_m, max_candidates)
    STATE["places"] = places
    return {"count": len(places)}

@tool("discover_places")
def discover_places_tool(
    lat: float,
    lon: float,
    radius_m: int = DEFAULTS["place_radius_m"],
    max_candidates: int = DEFAULTS["max_candidates"],
) -> dict:
    """Find restaurants/attractions near the destination and MERGE with any existing specific spots."""
    new_places = get_places(lat, lon, radius_m, max_candidates)

    # preserve any 'specific' items already in STATE["places"] (from your specific-need search)
    keep_specific = [p for p in STATE.get("places", []) if p.get("category") == "specific"]

    merged = keep_specific + (new_places or [])
    # de‑dup by (name, lat, lon)
    uniq, seen = [], set()
    for p in merged:
        key = (p.get("name",""), round(float(p.get("lat",0) or 0), 5), round(float(p.get("lon",0) or 0), 5))
        if key not in seen:
            seen.add(key); uniq.append(p)

    STATE["places"] = uniq
    return {
        "count": len(STATE["places"]),
        "added_now": len(new_places or []),
        "preserved_specific": len(keep_specific),
    }

# ===== Section 10: pick_and_route_tool — REPLACE =====
@tool("pick_and_route")
def pick_and_route_tool(
    origin_query: str,
    dest_query: str,
    top_k: int = DEFAULTS["top_k"],
    mode: str = DEFAULTS["transport_mode"],
    cost_per_km: float = DEFAULTS["cost_per_km"],
    time_value_per_hr: float = DEFAULTS["time_value_per_hr"],
) -> dict:
    """Select top-K stops and compute a low-cost route from origin → stops → destination."""
    g1 = geocode_nominatim(origin_query); g2 = geocode_nominatim(dest_query)
    if not g1 or not g2 or not STATE.get("places"):
        return {"ok": False, "error": "Missing geocodes or places."}

    origin = (g1[0], g1[1]); dest = (g2[0], g2[1])
    center = dest

    food, funs, scored = compute_fun_scores(STATE["places"], center)

    picks: List[Dict[str, Any]] = []

    # 1) If user asked for something, FORCE‑include the nearest 'specific' candidate
    want_specific = bool(STATE["inputs"].get("specific_need_free"))
    if want_specific:
        specifics = [p for p in scored if (p.get("category") == "specific") and (p.get("lat") is not None) and (p.get("lon") is not None)]
        if specifics:
            specifics.sort(key=lambda p: p.get("dist_from_center_km", 9e9))
            picks.append(specifics[0])

    # 2) Fill the rest by interleaving fun + food, avoiding dupes
    i = j = 0
    def _already(p):
        return any(haversine_km((p["lat"], p["lon"]), (q["lat"], q["lon"])) <= 0.01 for q in picks)

    while len(picks) < max(2, top_k) and (i < len(funs) or j < len(food)):
        if i < len(funs):
            cand = funs[i]; i += 1
            if cand.get("lat") is not None and cand.get("lon") is not None and not _already(cand):
                picks.append(cand)
        if len(picks) >= top_k: break
        if j < len(food):
            cand = food[j]; j += 1
            if cand.get("lat") is not None and cand.get("lon") is not None and not _already(cand):
                picks.append(cand)

    # 3) Route & cost
    route = plan_route(origin, dest, picks, mode=mode)
    total_km = route["total_distance_m"] / 1000.0
    total_hr = route["total_duration_s"] / 3600.0
    cost_est = total_km * max(0.0, float(cost_per_km)) + total_hr * max(0.0, float(time_value_per_hr))

    # 4) Human-readable order
    ordered = []
    all_nodes = [{"name": "Origin", "lat": origin[0], "lon": origin[1], "url": ""}] + picks + [{"name": "Destination", "lat": dest[0], "lon": dest[1], "url": ""}]
    for idx in route["order"]:
        if 0 <= idx < len(all_nodes):
            ordered.append(all_nodes[idx])

    STATE["route"] = {
        "mode": mode,
        "origin_label": g1[2],
        "destination_label": g2[2],
        "stops": picks,
        "order": ordered,
        "legs": route["legs"],
        "total_distance_km": round(total_km, 2),
        "total_duration_hr": round(total_hr, 2),
        "estimated_total_cost": round(cost_est, 2),
        "params": {"cost_per_km": cost_per_km, "time_value_per_hr": time_value_per_hr},
    }
    return {
        "ok": True,
        "summary": {
            "distance_km": STATE["route"]["total_distance_km"],
            "duration_hr": STATE["route"]["total_duration_hr"],
            "cost_est": STATE["route"]["estimated_total_cost"],
        },
    }

@tool("make_itinerary")
def make_itinerary_tool(preferences_json: str = "") -> str:
    """Generate a friendly Markdown itinerary based on the chosen route and sources."""
    prefs = preferences_json or "{}"

    md_content = llm_call([
        {"role": "system", "content": "You are a cheerful but concise travel concierge. Keep it practical, use bullet points."},
        {"role": "user", "content": f"""
Create a Markdown itinerary. Requirements:
- Start with a short overview.
- List the ordered stops with one-line reasons to visit and any known ratings if available.
- Include total distance, duration, and a small budget breakdown using the provided cost estimate.
- Add 6-8 quick tips (best time, booking, local transport, safety).
- Finish with 'Sources' linking the discovery guides and place URLs.

DATA (JSON):
inputs: {json.dumps(STATE["inputs"], ensure_ascii=False)}
route: {json.dumps(STATE["route"], ensure_ascii=False)}
guides: {json.dumps(STATE["search_sources"][:10], ensure_ascii=False)}
places_note: "Some places come from OSM and may not have ratings."
"""}
    ])

    # If LLM call fails, make a minimal placeholder
    if not md_content:
        md_content = "# Trip Itinerary\n\n(No AI-generated summary was available.)"

    STATE["itinerary_md"] = md_content

    # Always save Markdown and JSON, even on LLM failure
    with open(OUTPUT_MD, "w", encoding="utf-8") as f:
        f.write(STATE["itinerary_md"])
    with open(OUTPUT_JSON, "w", encoding="utf-8") as f:
        json.dump({
            "inputs": STATE["inputs"],
            "guides": STATE["search_sources"],
            "places": STATE["places"],
            "route": STATE["route"],
            "generated_at": datetime.datetime.utcnow().isoformat() + "Z"
        }, f, ensure_ascii=False, indent=2)

    return STATE["itinerary_md"]

#### **11) AGENT PROMPT**

In [163]:
SYSTEM_MSG = """You are a multi-tool travel agent. Follow the plan:
1) geo_destination(city) to validate destination.
2) discover_guides(city) for recent lists/tips (last 12 months intent).
3) discover_places(lat,lon) to fetch candidates.
4) pick_and_route(origin, destination, top_k, mode, cost_per_km, time_value_per_hr).
5) make_itinerary(preferences_json).

Only call tools. When the route exists, call make_itinerary to finish.
"""

agent_tools = [geo_destination_tool, discover_guides_tool, discover_places_tool, pick_and_route_tool, make_itinerary_tool]
prompt = ChatPromptTemplate.from_messages([
    ("system", SYSTEM_MSG),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}")
])

def run_agent(inputs: Dict[str,Any]):
    if not os.environ.get("GROQ_API_KEY"):
        raise RuntimeError("No GROQ_API_KEY; skipping tool-calling agent.")

    msg = textwrap.dedent(f"""
    User inputs (JSON):
    {json.dumps(inputs, ensure_ascii=False)}
    Begin the plan now starting with geo_destination.
    """).strip()

    llm = get_groq2_llm("llama3-70b-8192", temperature=0)
    agent = create_openai_tools_agent(llm, agent_tools, prompt)
    executor = AgentExecutor(agent=agent, tools=agent_tools, verbose=True, handle_parsing_errors=True, max_iterations=20)
    return executor.invoke({"input": msg})

#### **12) COLLECT USER INPUTS (INTERACTIVE)**


In [164]:
AGENT_NAME = "RouteForge"
print(f"🧭 {AGENT_NAME} here! craftsthe perfect route like a true navigator...")
origin = input("Enter your ORIGIN (address or city): ").strip()
final_destination = input("Enter your FINAL DESTINATION (address or city): ").strip()
city_for_guides = input("City/Area to explore (press Enter to reuse final destination): ").strip() or final_destination
mode = input("Transport mode [driving/walking/cycling] (default driving): ").strip().lower() or DEFAULTS["transport_mode"]
try:
    top_k = int(input(f"How many stops before the final destination? (default {DEFAULTS['top_k']}): ").strip() or DEFAULTS["top_k"])
except:
    top_k = DEFAULTS["top_k"]
try:
    radius_m = int(input(f"Search radius in meters? (default {DEFAULTS['place_radius_m']}): ").strip() or DEFAULTS["place_radius_m"])
except:
    radius_m = DEFAULTS["place_radius_m"]
try:
    cost_per_km = float(input(f"Cost per km (e.g., fuel, fare) (default {DEFAULTS['cost_per_km']}): ").strip() or DEFAULTS["cost_per_km"])
except:
    cost_per_km = DEFAULTS["cost_per_km"]
try:
    time_value_per_hr = float(input(f"Your time value per hour (default {DEFAULTS['time_value_per_hr']}): ").strip() or DEFAULTS["time_value_per_hr"])
except:
    time_value_per_hr = DEFAULTS["time_value_per_hr"]

STATE["inputs"] = {
    "origin": origin,
    "final_destination": final_destination,
    "city_for_guides": city_for_guides,
    "mode": mode,
    "top_k": top_k,
    "radius_m": radius_m,
    "cost_per_km": cost_per_km,
    "time_value_per_hr": time_value_per_hr
}

🧭 RouteForge here! craftsthe perfect route like a true navigator...
Enter your ORIGIN (address or city): darulsalam addison illinois 
Enter your FINAL DESTINATION (address or city): cumming georgia
City/Area to explore (press Enter to reuse final destination): 
Transport mode [driving/walking/cycling] (default driving): 
How many stops before the final destination? (default 8): 10
Search radius in meters? (default 4000): 
Cost per km (e.g., fuel, fare) (default 0.25): 5
Your time value per hour (default 5.0): 


In [165]:
STATE["inputs"]["specific_need_free"] = input("Anything specific to find? (e.g., 'cafe' — press Enter to skip: ").strip()
STATE["inputs"].get("specific_need_free")

Anything specific to find? (e.g., 'cafe' — press Enter to skip: i want to stop by a cafe and a flower shop


'i want to stop by a cafe and a flower shop'

In [166]:
import json as _json

def parse_specific_need_free(text: str):
    if not text:
        return []
    # try LLM (OpenAI primary, Groq fallback) to get precise JSON
    prompt = [
        {"role":"system","content":(
            "You convert a user's travel request into a compact JSON array of search intents. "
            "Allowed item types: 'place', 'category', 'area', 'osm'. "
            "Rules:\n"
            "- For a named business/venue, emit: {\"type\":\"place\",\"name\":\"...\"}\n"
            "- For categories, emit: {\"type\":\"category\",\"label\":\"cafe|pharmacy|restaurant|restroom|museum|park|mall|supermarket|...\",\"count\":<int optional>,\"cuisine\":\"italian|pakistani|...\" optional}\n"
            "- For an area inside the city (e.g., 'F-7', 'Liberty Market'), emit: {\"type\":\"area\",\"name\":\"...\"}\n"
            "- If you know the exact OSM tag, also add: {\"type\":\"osm\",\"key\":\"amenity|shop|tourism|leisure|...\",\"value\":\"toilets|pharmacy|supermarket|...\"}\n"
            "- Use 'restroom'→'toilets' for OSM vocabulary.\n"
            "Return ONLY valid JSON (no backticks, no text)."
        )},
        {"role":"user","content": f"Query: {text}\nReturn the JSON array now."}
    ]
    try:
        raw = llm_call(prompt)  # uses your Section 4 helpers
        obj = _json.loads(raw)
        if isinstance(obj, list):
            # normalize a bit
            for it in obj:
                if "type" in it: it["type"] = it["type"].lower()
                if "label" in it and isinstance(it["label"], str): it["label"] = it["label"].lower()
                if "name" in it and isinstance(it["name"], str): it["name"] = it["name"].strip()
            return obj
    except Exception as e:
        print("Parser LLM failed; will search literally:", e)
    # fallback: treat entire text as a single place string
    return [{"type":"place","name":text.strip()}]

STATE["inputs"]["specific_need_parsed"] = parse_specific_need_free(STATE["inputs"]["specific_need_free"])
print("Parsed intents:", STATE["inputs"]["specific_need_parsed"])


OpenAI failed, falling back to Groq: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}
Parsed intents: [{'type': 'category', 'label': 'cafe', 'count': 1}, {'type': 'category', 'label': 'flower shop', 'count': 1}]


#### **13) KICK OFF PIPELINE (Agent)**

In [167]:
import datetime as dt_module

try:
    geo = geocode_nominatim(city_for_guides)
    if not geo:
        raise RuntimeError("Could not geocode the exploration city/area.")
    dlat, dlon, city_label = geo
    center_xy = (dlat, dlon)

    try:
        city_bias_hit = geocode_in_city(STATE["inputs"].get("city_for_guides", city_for_guides), center_xy)
        if city_bias_hit:
            center_xy = (city_bias_hit[0], city_bias_hit[1])
    except Exception:
        pass

    parsed = STATE["inputs"].get("specific_need_parsed", [])
    specific_places = resolve_parsed_specific_requests(center_xy, radius_m, parsed, max_per_item=12)

    general_places = get_places(center_xy[0], center_xy[1], radius_m, DEFAULTS["max_candidates"])

    merged = (specific_places or []) + (general_places or [])
    dedup, seen = [], set()
    for p in merged:
        k = (p.get("name",""), round(float(p.get("lat",0) or 0),5), round(float(p.get("lon",0) or 0),5))
        if k not in seen:
            seen.add(k); dedup.append(p)
    STATE["places"] = dedup

    print("\nRunning the agent...")
    result = run_agent({
        "origin": origin,
        "destination": final_destination,
        "city": city_for_guides,
        "mode": mode,
        "top_k": top_k,
        "radius_m": radius_m,
        "cost_per_km": cost_per_km,
        "time_value_per_hr": time_value_per_hr,
        "specific_need": STATE["inputs"].get("specific_need_free","")
    })
    final_md = result.get("output") or result.get("final_output") or STATE.get("itinerary_md","")

except Exception as e:
    print("\nAgent failed or quota/rate-limit hit — deterministic fallback.", e)
    # minimal fallback
    if not STATE.get("places"):
        STATE["places"] = get_places(center_xy[0], center_xy[1], radius_m, DEFAULTS["max_candidates"])
    _ = pick_and_route_tool.invoke({
        "origin_query": origin,
        "dest_query": final_destination,
        "top_k": top_k,
        "mode": mode,
        "cost_per_km": cost_per_km,
        "time_value_per_hr": time_value_per_hr
    })
    final_md = make_itinerary_tool.invoke({"preferences_json": json.dumps(STATE["inputs"])})


Running the agent...


> Entering new AgentExecutor chain...

Invoking: `geo_destination` with `{'query': 'cumming georgia'}`


{'ok': True, 'lat': 34.2073196, 'lon': -84.1401926, 'label': 'Cumming, Forsyth County, Georgia, United States'}
Invoking: `discover_guides` with `{'city': 'Cumming, Forsyth County, Georgia, United States'}`


{'count': 25, 'samples': [{'title': 'THE 15 BEST Things to Do in Cumming (2025)', 'url': 'https://www.tripadvisor.com/Attractions-g34877-Activities-Cumming_Georgia.html', 'content': 'Top Attractions in Cumming ; 1. Sawnee Mountain Preserve · 4.6. (324) ; 2. Fowler Park · 4.8. (97) ; 3. Big Creek Greenway · 4.7. (145) ; 4. Cumming Aquatic Center.'}, {'title': 'Ultimate Local Guide To Cumming GA', 'url': 'https://www.goldpeachrealty.com/cumming-ga', 'content': 'As of 2024, the population of Cumming, Georgia, is approximately 7,300 people. The city is part of Forsyth County, which has seen significant growth in recent years.\n\n#### What are the top attract

#### **14) DONE — SHOW OUTPUT PATHS & PREVIEW**


In [168]:
print("\n==================== DONE ====================")
print(f"Markdown itinerary saved to: {os.path.abspath(OUTPUT_MD)}")
print(f"JSON (sources & route) saved to: {os.path.abspath(OUTPUT_JSON)}")

try:
    with open(OUTPUT_MD, "r", encoding="utf-8") as f:
        preview = "".join([next(f) for _ in range(40)])
    print("\n--- Report.md (preview) ---\n")
    print(preview)
except Exception as e:
    print("No preview available:", e)



==================== DONE ====================
Markdown itinerary saved to: /content/Report.md
JSON (sources & route) saved to: /content/trip_plan.json

--- Report.md (preview) ---

**Cumming, Georgia Road Trip Itinerary**

### Overview

Embark on a 4-day road trip from Darulsalam, Illinois to Cumming, Georgia, exploring the city's top attractions, cafes, and flower shops. This itinerary includes a mix of outdoor activities, cultural experiences, and delicious food.

### Ordered Stops

1. **Sawnee Mountain Preserve** (4.6/5) - Hike and enjoy scenic views of the surrounding mountains.
2. **Fowler Park** (4.8/5) - Visit the park's playground, picnic areas, and walking trails.
3. **Big Creek Greenway** (4.7/5) - Explore the 5.5-mile trail for hiking, biking, or walking.
4. **Cumming Aquatic Center** - Cool off during the summer months with a swim or splash pad.
5. **Tam's Backstage** (4.5/5) - Savor Southern cuisine and live music at this popular restaurant.
6. **Marie's Italian Deli** (